In [1]:
import os
import folium
import geopandas
import json
import pandas as pd

print(folium.__version__)



decoupage_regions = os.path.join('data', 'regions.geojson')

geo_json_data = json.load(open(decoupage_regions))

0.5.0


# GeoJSON and choropleth

**A few examples of how to do that with `folium`.**


## Using `GeoJson`


J'importe un découpage des nouvelles régions, issu de ce site : https://github.com/gregoiredavid/france-geojson

It is a classical GeoJSON `FeatureCollection` (see https://en.wikipedia.org/wiki/GeoJSON) of the form :

    {
        "type": "FeatureCollection",
        "features": [
            {
                "properties": {"code":"3", "nom": "Guyane"},
                "type": "Feature",
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [[[-87.359296, 35.00118], ...]]
                    }
                },
            {
                "properties": {"code":"11", "nom": "Île-de-France"},
                "type": "Feature",
                "geometry": {
                    "type": "MultiPolygon",
                    "coordinates": [[[[-131.602021, 55.117982], ... ]]]
                    }
                },
            ...
            ]
        }


# Constitution des jeux de données

In [2]:
med = pd.read_csv('data/med_tab1.csv', skiprows=4, encoding='windows-1252', error_bad_lines=False, )

In [3]:
med = med[1:]

In [4]:
med_regions = med[3:21]

In [5]:
med_regions

,SPECIALITE,Ensemble des spécialités d'exercice,Spécialistes,Anatomie et cytologie pathologiques,Anesthésie-réanimation,Biologie médicale,Cardiologie et maladies vasculaires,Chirurgie générale,Chirurgie maxillo-faciale et stomatologie,Chirurgie orthopédique et traumatologie,...,Pneumologie,Psychiatrie,Radiodiagnostic et imagerie médicale,Radiothérapie,Réanimation médicale,Recherche médicale,Rhumatologie,Santé publique et médecine sociale,Généralistes,Médecine générale
4,Alsace-Champagne-Ardenne-Lorraine,319.9,169.5,2.0,16.8,4.0,10.5,6.0,0.8,4.9,...,4.3,17.6,12.7,1.3,0.7,0.0,3.4,2.4,150.4,150.4
5,Aquitaine-Limousin-Poitou-Charentes,334.4,168.8,2.4,16.4,4.5,10.8,5.1,1.6,5.0,...,3.7,21.7,12.6,1.4,0.4,0.0,3.3,2.3,165.6,165.6
6,Auvergne-Rhône-Alpes,336.9,180.2,2.5,18.1,3.9,9.5,5.7,1.8,5.3,...,4.4,22.9,12.8,1.3,0.3,0.0,4.4,2.7,156.7,156.7
7,Bourgogne-Franche-Comté,294.6,148.9,1.7,14.4,3.6,8.1,4.6,1.2,5.1,...,4.4,17.3,10.4,1.2,0.5,0.0,3.1,2.6,145.7,145.7
8,Bretagne,315.7,158.2,1.9,15.2,3.9,9.3,4.0,0.6,4.5,...,4.6,18.9,10.7,1.2,0.2,0.0,3.9,2.2,157.5,157.5
9,Centre-Val de Loire,268.0,140.8,1.8,11.8,3.6,8.7,4.5,1.2,3.9,...,3.3,15.9,11.0,1.4,0.4,0.0,3.5,1.6,127.3,127.3
10,Corse,298.3,142.9,1.2,12.8,4.0,10.7,4.0,1.8,4.3,...,3.7,17.4,11.3,0.9,0.0,0.0,2.4,0.6,155.4,155.4
11,Guadeloupe,267.7,134.5,2.0,9.0,5.0,8.7,3.2,0.5,3.5,...,2.0,14.0,9.2,0.7,0.2,0.0,2.2,2.2,133.2,133.2
12,Guyane,227.1,86.4,1.2,10.2,3.1,3.1,3.9,0.0,2.0,...,0.8,5.5,5.5,0.0,0.8,0.0,0.0,5.5,140.6,140.6
13,Ile-de-France,398.3,248.8,3.3,19.3,6.4,13.7,6.7,2.7,5.1,...,5.4,36.7,17.1,1.5,1.0,0.0,5.5,4.0,149.5,149.5


In [6]:
conv_regions = pd.read_table('data/correspondance_regions.tsv')

In [7]:
# conv_regions['new_name_serialized'] = conv_regions['new_name'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
# conv_regions['old_name_serialized'] = conv_regions['former_name'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [8]:
conv_regions

,new_code,new_name,former_code,former_name
0,1,Guadeloupe,1,Guadeloupe
1,2,Martinique,2,Martinique
2,3,Guyane,3,Guyane
3,4,La Réunion,4,La Réunion
4,6,Mayotte,6,Mayotte
5,11,Île-de-France,11,Île-de-France
6,24,Centre-Val de Loire,24,Centre
7,27,Bourgogne-Franche-Comté,26,Bourgogne
8,27,Bourgogne-Franche-Comté,43,Franche-Comté
9,28,Normandie,23,Haute-Normandie


In [9]:
for i in geo_json_data['features']:
    print(i['properties'])

{'code': '11', 'nom': 'Île-de-France'}
{'code': '24', 'nom': 'Centre-Val de Loire'}
{'code': '27', 'nom': 'Bourgogne-Franche-Comté'}
{'code': '28', 'nom': 'Normandie'}
{'code': '32', 'nom': 'Hauts-de-France'}
{'code': '44', 'nom': 'Grand Est'}
{'code': '52', 'nom': 'Pays de la Loire'}
{'code': '53', 'nom': 'Bretagne'}
{'code': '75', 'nom': 'Nouvelle-Aquitaine'}
{'code': '01', 'nom': 'Guadeloupe'}
{'code': '02', 'nom': 'Martinique'}
{'code': '03', 'nom': 'Guyane'}
{'code': '04', 'nom': 'La Réunion'}
{'code': '06', 'nom': 'Mayotte'}
{'code': '76', 'nom': 'Occitanie'}
{'code': '84', 'nom': 'Auvergne-Rhône-Alpes'}
{'code': '93', 'nom': "Provence-Alpes-Côte d'Azur"}
{'code': '94', 'nom': 'Corse'}


Je fais ici la moyenne des densités des médecins tout en sachant que les anciennes régions n'avaient pas la même population. Il aurait fallu pondérer la moyenne par la population respective de chaque ancienne région mais j'ai décidé d'aller à l'essentiel.

In [10]:
# med_regions['SPECIALITE'] = med_regions['SPECIALITE'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')


In [11]:
# conv_regions.set_index('new_name_serialized').join(med_regions.set_index('SPECIALITE'))

In [12]:
med_regions['SPECIALITE'] = med_regions['SPECIALITE'].str.replace("Ile-de-France","Île-de-France").str.replace("Nord-Pas-de-Calais-Picardie","Hauts-de-France").str.replace("Alsace-Champagne-Ardenne-Lorraine","Grand Est").str.replace("Aquitaine-Limousin-Poitou-Charentes","Nouvelle-Aquitaine").str.replace("Languedoc-Roussillon-Midi-Pyrénées","Occitanie").str.replace("Provence-Alpes-Côte d’Azur","Provence-Alpes-Côte d'Azur")

/home/unixAlex/Envs/wsl3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
med_regions

,SPECIALITE,Ensemble des spécialités d'exercice,Spécialistes,Anatomie et cytologie pathologiques,Anesthésie-réanimation,Biologie médicale,Cardiologie et maladies vasculaires,Chirurgie générale,Chirurgie maxillo-faciale et stomatologie,Chirurgie orthopédique et traumatologie,...,Pneumologie,Psychiatrie,Radiodiagnostic et imagerie médicale,Radiothérapie,Réanimation médicale,Recherche médicale,Rhumatologie,Santé publique et médecine sociale,Généralistes,Médecine générale
4,Grand Est,319.9,169.5,2.0,16.8,4.0,10.5,6.0,0.8,4.9,...,4.3,17.6,12.7,1.3,0.7,0.0,3.4,2.4,150.4,150.4
5,Nouvelle-Aquitaine,334.4,168.8,2.4,16.4,4.5,10.8,5.1,1.6,5.0,...,3.7,21.7,12.6,1.4,0.4,0.0,3.3,2.3,165.6,165.6
6,Auvergne-Rhône-Alpes,336.9,180.2,2.5,18.1,3.9,9.5,5.7,1.8,5.3,...,4.4,22.9,12.8,1.3,0.3,0.0,4.4,2.7,156.7,156.7
7,Bourgogne-Franche-Comté,294.6,148.9,1.7,14.4,3.6,8.1,4.6,1.2,5.1,...,4.4,17.3,10.4,1.2,0.5,0.0,3.1,2.6,145.7,145.7
8,Bretagne,315.7,158.2,1.9,15.2,3.9,9.3,4.0,0.6,4.5,...,4.6,18.9,10.7,1.2,0.2,0.0,3.9,2.2,157.5,157.5
9,Centre-Val de Loire,268.0,140.8,1.8,11.8,3.6,8.7,4.5,1.2,3.9,...,3.3,15.9,11.0,1.4,0.4,0.0,3.5,1.6,127.3,127.3
10,Corse,298.3,142.9,1.2,12.8,4.0,10.7,4.0,1.8,4.3,...,3.7,17.4,11.3,0.9,0.0,0.0,2.4,0.6,155.4,155.4
11,Guadeloupe,267.7,134.5,2.0,9.0,5.0,8.7,3.2,0.5,3.5,...,2.0,14.0,9.2,0.7,0.2,0.0,2.2,2.2,133.2,133.2
12,Guyane,227.1,86.4,1.2,10.2,3.1,3.1,3.9,0.0,2.0,...,0.8,5.5,5.5,0.0,0.8,0.0,0.0,5.5,140.6,140.6
13,Île-de-France,398.3,248.8,3.3,19.3,6.4,13.7,6.7,2.7,5.1,...,5.4,36.7,17.1,1.5,1.0,0.0,5.5,4.0,149.5,149.5


In [15]:
m = folium.Map([47, 3], zoom_start=5)#, tiles='Stamen Watercolor')

tooltip = 'Click me!'

m.choropleth(
    geo_data=decoupage_regions,
    data=med_regions,
    columns=['SPECIALITE', 'Spécialistes'],
    key_on='feature.properties.nom',
    fill_color='RdYlGn',
    #threshold_scale=[3, 4, 9, 10]
    legend_name='Nombre de médeci',
    highlight=True,
    #tooltip=tooltip
)

m

In [ ]:
m.choropleth